<h1> Compare Assets - HaloPSA vs DattoRMM + Auvik </h1>

Review for Asset Ingestion into HaloPSA Gaps or Issues

# Import Modules and Define Globals

In [ ]:
# data import and file manipulation
import requests
from requests.structures import CaseInsensitiveDict
import json
import csv
import xlrd

#data conditioning
import pandas as pd
import numpy as np
import re
import datetime as dt

#data visualization
[REDACTED]/.pyplot as plt
import seaborn as sns

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# dictionary Directory
dictionary_dir = 'd:/git/example_infrastructure_data_dev/dictionaries'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

## Create env Variables

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

In [ ]:
# import and assign secrets from env.ini

dattormm_config = config['dattormm']
halopsa_config = config['halopsa']
auvik_config = config['auvik']

# Create Datto RMM DataFrame

## Create auth token

In [ ]:
# call token api url
token_uri = f"{dattormm_config['base_uri']}/auth/oauth/token"


# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = dattormm_config['api_key']
data['password'] = dattormm_config['api_secret']

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']

## Create DataFrame via API Call Iteration


In [ ]:

## Create Devices DataFrame
# request content response
request_url = f"{dattormm_config['base_uri']}/api/v2/account/devices"

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


# iterate and combine remaining pages
df_dattormm = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)

    df_current_page = pd.DataFrame(c_dict['devices'])
    df_dattormm = pd.concat([df_dattormm, df_current_page], ignore_index=False)

# Create HaloPSA DataFrame


## Create Auth Token

In [ ]:
# call token api url
token_uri = f"{halopsa_config['base_uri']}/auth/token?tenant=example"

# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'


# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'client_credentials'
data['client_id'] = halopsa_config['client_id']
data['client_secret'] = halopsa_config['client_secret']
data['scope'] = 'all'

# request content response
resp = requests.post(token_uri, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']

## Create Asset DataFrame

In [ ]:
# request content response
request_url = f"{halopsa_config['base_uri']}/api/Asset"

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


# iterate and combine remaining pages
df_halopsa = pd.DataFrame(c_dict['assets'])
try:
    while c_dict['pageDetails']['nextPageUrl']:
        next_page = c_dict['pageDetails']['nextPageUrl']
        resp = requests.get(next_page, headers=headers, data=data)
        content = resp.content.decode('utf-8')
        c_dict = json.loads(content)

        df_current_page = pd.DataFrame(c_dict['assets'])
        df_halopsa = pd.concat([df_halopsa, df_current_page], ignore_index=False)
except:
    print(f'All assets on first page.  Total Assets: {c_dict["record_count"]}')

# Create Auvik DataFrame

## Verify permissions in Auvik


In [ ]:
# verify permissions URL

request_uri = f"{auvik_config['base_uri']}/authentication/verify"

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f"Basic {auvik_config['api_key']}"
#headers['auth'] = f'{username,basic_auth}'

# construct req body
data = ""

# request content response (200) is GOOD
resp = requests.get(request_uri, auth=(auvik_config['username'],auvik_config['api_key']), headers=headers)
print(resp.status_code)
print(resp)

## Create DataFrame
### Iterate through devices

In [ ]:
# request uri
request_uri = f"{auvik_config['base_uri']}/inventory/device/info"

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f"Basic {auvik_config['api_key']}"

# construct req body
data = CaseInsensitiveDict()

# request content response
resp = requests.get(request_uri,  auth=(auvik_config['username'],auvik_config['api_key']), headers=headers)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

# count pages
total_pages = 0

df_auvik = pd.DataFrame(c_dict['data'])
total_pages += 1

while c_dict['links']['next']:
    next_page = c_dict['links']['next']
    resp = requests.get(next_page,  auth=(auvik_config['username'],auvik_config['api_key']), headers=headers)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)
    df_current_page = pd.DataFrame(c_dict['data'])
    df_auvik = pd.concat([df_auvik,df_current_page],ignore_index=False)
    try:
        print(c_dict['links']['next'])
        total_pages += 1
    except:
        print('*'* 200)
        print('End of API Pages')
        print(f'Total Pages: {total_pages}')
        break


## Create Columns from Dictionaries

### Attributes Column Breakout
['ipAddresses'],['deviceName'],['deviceType'],['makeModel'],['vendorName'],['softwareVersion'],['ipAddresses'],['serialNumber'],['description'],['firmwareVersion'],['lastModified'],['lastSeenTime'],['onlineStatus'])

In [ ]:
# ipAddresses
def ipAddresses(attributes):
    return attributes['ipAddresses']


df_auvik['ipAddresses'] = df_auvik['attributes'].apply(ipAddresses)

# deviceName
def deviceName(attributes):
    return attributes['deviceName']


df_auvik['deviceName'] = df_auvik['attributes'].apply(deviceName)

# makeModel
def makeModel(attributes):
    return attributes['makeModel']


df_auvik['makeModel'] = df_auvik['attributes'].apply(makeModel)

# vendorName
def vendorName(attributes):
    return attributes['vendorName']


df_auvik['vendorName'] = df_auvik['attributes'].apply(vendorName)

# softwareVersion
def softwareVersion(attributes):
    return attributes['softwareVersion']


df_auvik['softwareVersion'] = df_auvik['attributes'].apply(softwareVersion)

# serialNumber
def serialNumber(attributes):
    return attributes['serialNumber']


df_auvik['serialNumber'] = df_auvik['attributes'].apply(serialNumber)

# description
def description(attributes):
    return attributes['description']


df_auvik['description'] = df_auvik['attributes'].apply(description)


# firmwareVersion
def firmwareVersion(attributes):
    return attributes['firmwareVersion']


df_auvik['firmwareVersion'] = df_auvik['attributes'].apply(firmwareVersion)


# lastModified
def lastModified(attributes):
    return attributes['lastModified']


df_auvik['lastModified'] = df_auvik['attributes'].apply(lastModified)


# lastSeenTime
def lastSeenTime(attributes):
    return attributes['lastSeenTime']


df_auvik['lastSeenTime'] = df_auvik['attributes'].apply(lastSeenTime)


# onlineStatus
def onlineStatus(attributes):
    return attributes['onlineStatus']


df_auvik['onlineStatus'] = df_auvik['attributes'].apply(onlineStatus)




# drop attributes column after breakout
df_auvik.drop('attributes', axis=1, inplace=True)

### Relationship Column Breakout

In [ ]:
# tenant ID
def tenantID(relationships):
    return relationships['tenant']['data']['id']


df_auvik['tenantID'] = df_auvik['relationships'].apply(tenantID)


# domainPrefix
def domainPrefix(relationships):
    return relationships['tenant']['data']['attributes']['domainPrefix']


df_auvik['domainPrefix'] = df_auvik['relationships'].apply(domainPrefix)




# drop relationships column after breakout
df_auvik.drop('relationships', axis=1, inplace=True)

## Convert Timezone to DateTime UTC

In [ ]:
df_data_timefix = pd.to_datetime(df_auvik['lastSeenTime'],unit='ns',errors='coerce')
df_auvik['lastSeenTime'] = df_data_timefix.values.astype('datetime64[s]')
df_data_timefix = pd.to_datetime(df_auvik['lastModified'],unit='ns',errors='coerce')
df_auvik['lastModified'] = df_data_timefix.values.astype('datetime64[s]')

# Data Shaping

## Standardize Client Names

In [ ]:
df = pd.read_csv(f'{dictionary_dir}/client_name_standardization.dict',delimiter='\t')
client_rename_dict = {}
for index, row in df.iterrows():
    [REDACTED] = row['[REDACTED]']
    currentName = row['currentName']
    client_rename_dict[[REDACTED]] = currentName

In [ ]:
def client_names(c_name):
    dict_length = len(client_rename_dict)
    for k, v in client_rename_dict.items():
        try:
            result = re.sub(k, v, c_name)
            if result != c_name:
                print(f'Keyword found: {k}')
                print(f'Replacment value: {v}')
                print('\n')
                return v
                break
        except Exception as e:
            print(e)
            break
    return c_name

In [ ]:
df_auvik['clientName'] = df_auvik['domainPrefix'].apply(client_names)
df_halopsa['clientName'] = df_halopsa['client_name'].apply(client_names)
df_dattormm['clientName'] = df_dattormm['siteName'].apply(client_names)

## Breakout HaloPSA DataFrame into [['isAuvik','isDattoRMM']]

In [ ]:
def is_auvik(auvik_device_id):
    if auvik_device_id:
        return 1
    else:
        return 0

In [ ]:
def is_datto(datto_id):
    if datto_id:
        return 1
    else:
        return 0

In [ ]:
df_halopsa['isAuvik'] = df_halopsa['auvik_device_id'].apply(is_auvik)
df_halopsa['isDattoRMM'] = df_halopsa['datto_id'].apply(is_auvik)

# Compare df_halopsa integration captured id's with id captured from product portal api pull

## DattoRMM Compare

In [ ]:
not_found = ~df_dattormm['uid'].isin(df_halopsa[df_halopsa['isDattoRMM'] == 1]['datto_id'].tolist())
found = df_dattormm['uid'].isin(df_halopsa[df_halopsa['isDattoRMM'] == 1]['datto_id'].tolist())

df_dattormm.loc[not_found, 'inHaloPSA'] = 0
df_dattormm.loc[found, 'inHaloPSA'] = 1
df_dattormm['inHaloPSA'] = df_dattormm['inHaloPSA'].astype('int64')

In [ ]:
df_dattormm['inHaloPSA'].value_counts()

In [ ]:
df_dattormm.to_csv(export_folder + 'dattormm_halopsa_assets_comparison_' + str(current_time) + '[REDACTED]/.csv',index=False)

## Auvik Compare

In [ ]:
not_found = ~df_auvik['id'].isin(df_halopsa[df_halopsa['isAuvik'] == 1]['auvik_device_id'].tolist())
found = df_auvik['id'].isin(df_halopsa[df_halopsa['isAuvik'] == 1]['auvik_device_id'].tolist())

df_auvik.loc[not_found, 'inHaloPSA'] = 0
df_auvik.loc[found, 'inHaloPSA'] = 1
df_auvik['inHaloPSA'] = df_auvik['inHaloPSA'].astype('int64')

In [ ]:
df_auvik['inHaloPSA'].value_counts()

In [ ]:
df_auvik.to_csv(export_folder + 'auvik_halopsa_assets_comparison_' + str(current_time) + '[REDACTED]/.csv',index=False)

## Extra Testing Code

In [ ]:
df_auvik[df_auvik['inHaloPSA'] == 0]

In [ ]:
df_halopsa[df_halopsa['auvik_device_id'] == "MzY1NzQzMjgxMTg1MDk1ODE0LDQyMTU3NDI3NzU2NDA0MDIxNA"]

In [ ]:
df_halopsa[df_halopsa['datto_id'] == "87b374c7-77c1-a843-8014-ccf1dfd37d99"]